# Setup Enviorment

In [1]:
from Preprocessing.selfsupervised_data_praparation import plot_samples, get_dataset_list
from Preprocessing.generate_embeddings import generate_embeddings_df, save_embeddings_as_csv
from tensorflow.keras import Sequential
from tensorflow.keras.applications import MobileNetV2, VGG16, ResNet50V2

import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd 

In [2]:
# Path to Dataset
path = '../Dataset_10_best_cities/'
model_name = 'vit' #'VGG16' #'MobileNetV2' # 'vit' # 'autoencoder' #'variational_autoencoder'
latent_dim = 1024

target_size = (224, 224, 1)
model_input = (224, 224, 3)

# Model path
model_path = f'Weights/{model_name}_{target_size[0]}_{latent_dim}.h5'  

## Read dataset

In [3]:
#plot_samples(path)

In [4]:
image_list = get_dataset_list(path, show_dirs=True, head=0)

Images in directories: 
54001
41001
5001
50001
68001
8001
23001
76001
73001
5360


## Load Model

In [5]:
tf.random.set_seed(0)

if model_name == 'variational_autoencoder':
    from Models import Variational_Autoencoder
    model = Variational_Autoencoder.get_Variational_Autoencoder(model_path=model_path, backbone=True, target_size = model_input, latent_dim = latent_dim, lr=0.0001)
elif model_name == 'autoencoder':
    from Models import Autoencoder
    model = Autoencoder.get_Autoencoder(model_path=model_path, backbone=True, target_size = model_input, latent_dim = latent_dim)
elif model_name == 'vit':
    from Models import ViT
    model = ViT.get_vit_backbone(model_input)
elif model_name == 'MobileNetV2':
    cnn = MobileNetV2(input_shape=model_input, include_top=False, weights='imagenet')
elif model_name == 'VGG16': # min depth
    cnn = VGG16(input_shape=model_input, include_top=False, weights='imagenet')
elif model_name == 'ResNet50V2':
    cnn = ResNet50V2(input_shape=model_input, include_top=False, weights='imagenet') 

if model_name in ['MobileNetV2', 'VGG16', 'ResNet50V2']:
    model = Sequential()
    model.add(cnn)
    model.add(tf.keras.layers.GlobalAveragePooling2D())

    # freeze:
    for layer in model.layers:
        layer.trainable = False
        
model.summary()

2023-03-05 18:25:08.791559: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-05 18:25:08.791698: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-05 18:25:08.791761: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (9f7bdfccc75e): /proc/driver/nvidia/version does not exist
2023-03-05 18:25:08.792516: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFViTModel.

All the layers of TFViT

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
permute (Permute)            (None, 3, 224, 224)       0         
_________________________________________________________________
vit (TFViTMainLayer)         TFBaseModelOutputWithPool 86389248  
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 768)               0         
Total params: 86,389,248
Trainable params: 0
Non-trainable params: 86,389,248
_________________________________________________________________


## Generate embeddings

In [ ]:
for band in range(12):
    if band+1 < 4:
        print(f'Generating Embeddings for the band {band+1} with the model {model_name}...')
        print('Done!')
        continue
    print(f'Generating Embeddings for the band {band+1} with the model {model_name}...')
    # Embeddings path
    embeddings_path = f'Embeddings/{model_name}_Per_Band/{target_size[0]}_band{band+1}.csv'
    # Generate Embeddings
    embeddings = generate_embeddings_df(image_list=image_list, model=model, target_size=target_size, BAND=band)
    # Save Embeddings
    save_embeddings_as_csv(df=embeddings, path=embeddings_path)
    print('Done!')

Generating Embeddings for the band 1 with the model vit...
Done!
Generating Embeddings for the band 2 with the model vit...
Done!
Generating Embeddings for the band 3 with the model vit...
Done!
Generating Embeddings for the band 4 with the model vit...


2023-03-05 18:25:21.654517: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Done!
Generating Embeddings for the band 5 with the model vit...
Done!
Generating Embeddings for the band 6 with the model vit...
Done!
Generating Embeddings for the band 7 with the model vit...


In [ ]:
pd.read_csv(embeddings_path)